# Introduction

# Read Data

Make sure to have the data locally locally saved as `/data`.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

In [ ]:
# Load in Artist CSV data
#print(os.listdir("./"))

# get the directory 
main_direc = os.getcwd()
print(main_direc)

artist_csv_loc = os.path.join(main_direc, 'data/artists.csv')
artists = pd.read_csv(artist_csv_loc)
print(artists.shape)
print(artists)

# Data Processing

The following approach is a modified version of the approach from [DeepArtists: Identify Artist from Art](https://www.kaggle.com/supratimhaldar/deepartist-identify-artist-from-art).

In [ ]:
# Sortting the artists by number of paintings
artists = artists.sort_values(by=['paintings'], ascending=False)
# print(artists)

In [ ]:
# Creating a dataframe with the top 10 artists by number of paintings
artists_top = artists.head(10)
artists_top

## Exploring data and creating a DataFrame of all image paths with their associated artists

In [ ]:
# Images directories, cut down is just top 10 artists
images_dir = os.path.join(main_direc, 'data/images/images')
cut_down_images_dir = os.path.join(main_direc, 'data/limited_images')

In [ ]:
# Images DataFrame
artists_top_name = artists_top['name'].str.replace(' ', '_').values

images_df = pd.DataFrame()
for name in artists_top_name:
    #print(glob.glob(images_dir + "/" + name + '/*'))

    # Method 1:
    #
    #images_df = images_df.append(pd.DataFrame(data={'Path': glob.glob('../cs254-final-project/data/images/images/' + name + '/*'), 'Name': name}), ignore_index=True)

    # Method 2:
    #
    images_df = pd.concat([images_df, pd.DataFrame(data={'Path': ["data/images/images/" + name + "/" + os.path.basename(x) for x in glob.glob(images_dir + "/" + name + '/*', )], 'Name': name})], ignore_index=True)

images_df

In [ ]:
# Converting the Images DataFrame to a .csv
# images_df.to_csv(os.path.join(main_direc, 'data/images.csv'), index=True)

## Load In Data

In [ ]:
# Load data
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array,load_img
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow as tf

def load_data(artists_csv_loc='data/artists.csv', images_dir='data/images/images'):
    artists = pd.read_csv(artists_csv_loc)
    print(artists.shape)
    # print(artists)

    # Creating a dataframe with the top 10 artists by number of paintings
    artists_top = artists.sort_values(by=['paintings'], ascending=False)
    print(artists_top)

    # Images
    artists_dir = os.listdir(images_dir) # Files are named after each artists

    # Images DataFrame
    artists_top_name = artists_top['name'].str.replace(' ', '_').values

    images_df = pd.DataFrame()
    for name in artists_top_name:
        images_df = pd.concat([images_df, pd.DataFrame(data={'Path': glob.glob('data/images/images/' + name + '/*'), 'Name': name})], ignore_index=True)

    print(images_df)

    # Create Generator


    BATCH_SIZE = 64

    # image dimensions?
    img_width, img_height = 277, 277

    train_df = images_df.sample(frac=0.8, random_state=200)
    test_df = images_df.drop(train_df.index)

    if K.backend.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)

    # instantiate neural network

    # Train

    train_generator = ImageDataGenerator(rescale=1.0 / 255,
                                        rotation_range=20,
                                        zoom_range=0.05,
                                        width_shift_range=0.05,
                                        height_shift_range=0.05,
                                        shear_range=0.05,
                                        horizontal_flip=True,
                                        fill_mode="nearest",
                                        validation_split=0.15,
                                        preprocessing_function=preprocess_input
                                        )

    test_generator = ImageDataGenerator(rescale=1.0 / 255, preprocessing_function=preprocess_input)

    train_gen = train_generator.flow_from_dataframe(
        train_df,
        shuffle=True,
        x_col='Path',
        y_col='Name',
        class_mode='categorical',
        subset="training",
        batch_size=BATCH_SIZE,
        target_size=(img_width, img_height),
        seed=42
    )

    valid_gen = train_generator.flow_from_dataframe(
        train_df,
        subset="validation",
        shuffle=True,
        x_col='Path',
        y_col='Name',
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        target_size=(img_width, img_height),
        seed=42
    )

    test_gen = test_generator.flow_from_dataframe(
        test_df,
        x_col='Path',
        batch_size=1,
        shuffle=False,
        class_mode=None,
        target_size=(img_width, img_height)
    )

    return train_gen, valid_gen, test_gen

train_gen, valid_gen, test_gen = load_data()

## BASELINE MODEL
The following model is a simple convolutional neural network using the keras
Sequential API.

In [ ]:
# BUILD MODEL
num_classes = 10

sequential_model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

# compile model
sequential_model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)


In [ ]:
# FITTING THE MODEL
sequential_model.fit(
  train_gen,
  validation_data=valid_gen,
  epochs=3
)


## The following model is Clay Kaufmann's model.
This model takes advantage of transfer learning, using Inception v3.

In [ ]:
# import inception
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import tensorflow.keras as K

# set the input for VGG
inp = Input(shape=(img_height,img_width,3))

# load model
base_model = InceptionV3(include_top=False, input_tensor=inp, pooling='max', weights='imagenet')

# set base model to not be trainable
base_model.trainable = False

final_model = tf.keras.Sequential()
final_model.add(base_model)

final_model.add(K.layers.Flatten())
final_model.add(K.layers.BatchNormalization())
final_model.add(Dense(256, activation='relu'))
final_model.add(K.layers.Dropout(0.6))
final_model.add(Dense(num_classes, activation='softmax'))

final_model.summary()

# compile model
final_model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['accuracy']
)

In [ ]:
# train model
final_model.fit(
  train_ds,
  validation_data=vals_ds,
  epochs=3
)